## Optimizing DBSCAN's hyperparameters minPts, eps on the Two Moons dataset

To reproduce this experiment, first delete or remove the database DBSCAN_two_moons_HPO.db that is stored in the results/optimization folder. Otherwise, the TPE optimization will use the 100 trials stored in the existing DBSCAN_two_moons_HPO.db and continue with the 101th trial up to the 200th trial. 

In [1]:
import sys  
sys.path.insert(0, '../..')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import optuna
import numpy as np
import pandas as pd

from sklearn.datasets import make_blobs, make_moons
from sklearn.cluster import KMeans, DBSCAN, SpectralClustering, MeanShift, estimate_bandwidth
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import adjusted_mutual_info_score as AMI
from rock import ROCK

import logging
import sys

In [4]:
def objective(trial):
    seed = 0

    results = []

    n_samples = 1000
    jitter = 0.15

    min_pts = trial.suggest_int('min_pts', 2, 100)
    eps = trial.suggest_int('eps', 1, 10) * 0.1

    datasets = []
    n_centers = None
    
    for s in range(seed, seed+10):
        datasets.append(make_moons(n_samples=n_samples, noise=jitter,
            shuffle=True, random_state=s))
        
    step = 0
    for dataset in datasets: 
        run = {}
        run['step'] = step
        run['n_samples'] = n_samples
        run['jitter'] = jitter
        scaler = StandardScaler()
        data = scaler.fit_transform(dataset[0])

        dbscan = DBSCAN(eps=eps, min_samples=min_pts).fit(data).labels_
        run['eps'] = eps
        run['min_pts'] = min_pts

        gt = dataset[1]
        results.append(AMI(dbscan, gt))
        run['DBSCAN'] = AMI(dbscan, gt)
        step += 1
        experiment.append(run)

    return np.mean(results)

In [5]:
experiment = []

In [6]:
study_name = 'DBSCAN_two_moons_HPO'

In [7]:
storage_name = f'sqlite:///../../results/optimization/{study_name}.db'
sampler = optuna.samplers.TPESampler(seed=0)
study = optuna.create_study(study_name=study_name, storage=storage_name, sampler=sampler, load_if_exists=True,  direction="maximize")

[I 2021-06-01 18:50:50,920] A new study created in RDB with name: DBSCAN_two_moons_HPO


In [8]:
study.optimize(objective, n_trials=100)

[I 2021-06-01 18:50:58,442] Trial 0 finished with value: 0.0 and parameters: {'min_pts': 56, 'eps': 8}. Best is trial 0 with value: 0.0.
[I 2021-06-01 18:50:58,658] Trial 1 finished with value: 0.0001820904250210742 and parameters: {'min_pts': 61, 'eps': 6}. Best is trial 1 with value: 0.0001820904250210742.
[I 2021-06-01 18:50:58,859] Trial 2 finished with value: 7.672985006701068e-16 and parameters: {'min_pts': 43, 'eps': 7}. Best is trial 1 with value: 0.0001820904250210742.
[I 2021-06-01 18:50:59,090] Trial 3 finished with value: 0.0 and parameters: {'min_pts': 45, 'eps': 9}. Best is trial 1 with value: 0.0001820904250210742.
[I 2021-06-01 18:50:59,259] Trial 4 finished with value: 0.0 and parameters: {'min_pts': 97, 'eps': 4}. Best is trial 1 with value: 0.0001820904250210742.
[I 2021-06-01 18:50:59,444] Trial 5 finished with value: 0.0009439327152330999 and parameters: {'min_pts': 80, 'eps': 6}. Best is trial 5 with value: 0.0009439327152330999.
[I 2021-06-01 18:50:59,675] Trial 

[I 2021-06-01 18:51:06,767] Trial 46 finished with value: 0.5842809606119299 and parameters: {'min_pts': 39, 'eps': 4}. Best is trial 14 with value: 0.8223943608040875.
[I 2021-06-01 18:51:06,936] Trial 47 finished with value: -0.00041245973435900183 and parameters: {'min_pts': 2, 'eps': 3}. Best is trial 14 with value: 0.8223943608040875.
[I 2021-06-01 18:51:07,121] Trial 48 finished with value: 0.2843963826403294 and parameters: {'min_pts': 24, 'eps': 2}. Best is trial 14 with value: 0.8223943608040875.
[I 2021-06-01 18:51:07,274] Trial 49 finished with value: 0.0 and parameters: {'min_pts': 100, 'eps': 1}. Best is trial 14 with value: 0.8223943608040875.
[I 2021-06-01 18:51:07,472] Trial 50 finished with value: 0.7039805622386408 and parameters: {'min_pts': 35, 'eps': 3}. Best is trial 14 with value: 0.8223943608040875.
[I 2021-06-01 18:51:07,653] Trial 51 finished with value: 0.7881525424337006 and parameters: {'min_pts': 30, 'eps': 3}. Best is trial 14 with value: 0.82239436080408

[I 2021-06-01 18:51:15,078] Trial 92 finished with value: 0.829386229002961 and parameters: {'min_pts': 42, 'eps': 4}. Best is trial 80 with value: 0.8300476666603241.
[I 2021-06-01 18:51:15,263] Trial 93 finished with value: 0.8300476666603241 and parameters: {'min_pts': 41, 'eps': 4}. Best is trial 80 with value: 0.8300476666603241.
[I 2021-06-01 18:51:15,448] Trial 94 finished with value: -0.0006429245958157786 and parameters: {'min_pts': 42, 'eps': 5}. Best is trial 80 with value: 0.8300476666603241.
[I 2021-06-01 18:51:15,626] Trial 95 finished with value: 0.8300476666603241 and parameters: {'min_pts': 41, 'eps': 4}. Best is trial 80 with value: 0.8300476666603241.
[I 2021-06-01 18:51:15,795] Trial 96 finished with value: 0.8298539801615614 and parameters: {'min_pts': 40, 'eps': 4}. Best is trial 80 with value: 0.8300476666603241.
[I 2021-06-01 18:51:15,980] Trial 97 finished with value: 0.8300476666603241 and parameters: {'min_pts': 41, 'eps': 4}. Best is trial 80 with value: 0.8

In [9]:
results = pd.DataFrame(experiment)

In [10]:
results.to_csv('../../results/optimization/DBSCAN_two_moons_HPO.csv')

In [11]:
results.iloc[80 * 10 : 81 * 10].mean()['DBSCAN']

0.8300476666603241

In [12]:
results.iloc[80 * 10 : 81 * 10].std()['DBSCAN']

0.024435737741039746